In [1]:
import json

In [2]:
with open('/home/roadcom/workspace/myflow/mydata.json', 'r') as fr:
    graph_data = json.load(fr)

In [3]:
# Node 실행 순서 parsing
nodes = {}
for node in graph_data['operators']:
    nodes[node] = []
for k, edge in graph_data['links'].items():
    nodes[str(edge['fromOperator'])].append(str(edge['toOperator']))    

In [4]:
# Node 실행시 입력으로 들어온 node의 파라미터 parsing
nodes_ins = {}
for k, edge in graph_data['links'].items():
    
    to_node = str(edge[str('toOperator')])
    if to_node not in nodes_ins:
        nodes_ins[to_node] = {}
    
    nodes_ins[to_node][edge['toConnector']] = (str(edge['fromOperator']), edge['fromConnector'])

In [5]:
nodes = {}
for node in graph_data['operators']:
    nodes[node] = {}
    
for k, edge in graph_data['links'].items():
    nodes[str(edge['fromOperator'])][str(edge['toOperator'])] = edge

In [6]:
def bfs(graph, start):
    visited = []
    queue = [start]

    while queue:
        n = queue.pop(0)
        if n not in visited:
            visited.append(n)
            
            for k in graph[n]:
                if k not in visited:
                    queue.append(k)
    return visited

In [7]:
def find_path(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        if node not in path:
            newpath = find_path(graph, node, end, path)
            if newpath: return newpath
    return None

In [8]:
def find_shortest_path(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if start not in graph:
        return None
    shortest = None
    for node in graph[start]:
        if node not in path:
            newpath = find_shortest_path(graph, node, end, path)
            if newpath:
                if not shortest or len(newpath) < len(shortest):
                    shortest = newpath
    return shortest

In [9]:
print('shortest path')
for n in find_shortest_path(nodes, '0', '4'):
    print(graph_data['operators'][n]['properties']['title'], end="▶")

print('\nbfs path')
for n in bfs(nodes, '0'):
    print(graph_data['operators'][n]['properties']['title'], end="▶")    

shortest path
SOME1▶Padding▶Concate▶DNN▶
bfs path
SOME1▶Resize▶Padding▶Concate▶Concate▶DNN▶

In [10]:
def some1(): return 'some1'
def some2(): return 'some2'
def some3(): return 'some3'
def some4(): return 'some4'

def resize(input_0): return 'resize'
def padding(input_0): return 'padding'
def crop(input_0): return 'crop'
def concate(input_0, input_1): return 'concate ' + input_0 + input_1
def cnn(input_0): return 'cnn'
def boosting(input_0): return 'boosting'
def dnn(input_0): return 'DNN'


funtion_pointer = {
    "SOME1" : some1,
    "SOME2" : some2,
    "SOME3" : some3,
    "SOME4" : some4,
    "Resize" : resize,
    "Padding" : padding,
    "Crop": crop,
    "Concate": concate,
    "CNN": cnn,
    "DNN": dnn,
    "boosting": boosting
}

In [11]:
lookup_table = {}

for n in bfs(nodes, '0'):
    
    title = graph_data['operators'][n]['properties']['title']
    
    ins = graph_data['operators'][n]['properties']['inputs'].keys()
    outs = graph_data['operators'][n]['properties']['outputs'].keys()
    
    for k in outs:
        operator_output = '{}|{}'.format(n, k)
        lookup_table[n] = {k: '{}_{}_SOME'.format(n, k)}
    
    function_params = {}
    
    for k in ins:
        from_node, from_label = nodes_ins[n][k]
        function_params[k] = lookup_table[from_node][from_label]
        
    print(funtion_pointer[title](**function_params))

some1
resize
padding
concate1_output_0_SOME2_output_0_SOME
concate3_output_0_SOME2_output_0_SOME
DNN
